In [12]:
import pandas as pd
from pandasql import sqldf, load_meat, load_births
pysqldf = lambda q: sqldf(q)
import numpy as np
import duckdb

If you do not have duckdb installed in your environment, just execute the following code: 

!pip install duckdb

# SQL OLAP EXPRESSION SYNTAX REFERENCE
SQL is a powerful language for data analysis. This series will delve into Online Analytical Processing (OLAP) expressions, a valuable extension that unlocks deeper insights from your data.

We'll leverage the comprehensive Oracle's [OLAP Expression Syntax Reference as a guide](https://docs.oracle.com/cd/E11882_01/olap.112/e23381/olap_functions.htm#OLAXS169)

**What are OLAP Expressions?**

Think of them as supercharged SQL expressions. 
They combine familiar elements like analytic functions, operators, and single-row functions to perform advanced calculations and analyze trends within your data.

**Why Learn Them?**

If you're already comfortable with SQL, the good news is that a large portion of OLAP syntax will feel intuitive. Mastering these expressions empowers you to:
- Calculate rankings and shares
- Analyze trends over time (e.g., moving averages)
- Compare performance across different dimensions
- Perform complex aggregations (e.g., year-to-date)

**What to Expect:**
This series will be structured into two main sections:
- **OLAP Functions:** We'll explore various functions designed for powerful data manipulation.
- **Row Functions:** Dive deep into functions that operate on specific rows within your data set.

Every week I will be posting two commands and adding them in their corresponding section. 

## 0. Defining the sample (and dummy) table
To exemplify each command, some dummy data is generated to work with.

In [4]:
# Define the data for the DataFrame
data = {
    'Date': ['1/1/22', '1/1/22', '1/1/22', '1/1/22', '2/1/22','5/1/22', '6/1/22', '7/1/22', '8/1/22', '9/1/22', '10/1/22'],
    'Expenses': [143.5, 25, np.nan, 24, 3.75, 20, 5, 100, 24.5, 2, 100],
    'Type': ['Gift', 'Clothes', 'Gaming', 'Clothes', 'Coffee', 'Gift', 'Clothes', 'Gaming', 'Clothes', 'Coffee', "Book"]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')

# Generate an extended dataframe with day, month and year. 
df_ext = df

# Extract the 'Day' and 'Year' from the 'Date' column
df_ext['Day'] = df['Date'].dt.day
df_ext['Month'] = df['Date'].dt.month
df_ext['Year'] = df['Date'].dt.year

# Create a DataFrame
df_ext


,Date,Expenses,Type,Day,Month,Year
0,2022-01-01,143.50,Gift,1,1,2022
1,2022-01-01,25.00,Clothes,1,1,2022
2,2022-01-01,NaN,Gaming,1,1,2022
3,2022-01-01,24.00,Clothes,1,1,2022
4,2022-01-02,3.75,Coffee,2,1,2022
5,2022-01-05,20.00,Gift,5,1,2022
6,2022-01-06,5.00,Clothes,6,1,2022
7,2022-01-07,100.00,Gaming,7,1,2022
8,2022-01-08,24.50,Clothes,8,1,2022
9,2022-01-09,2.00,Coffee,9,1,2022


## 1. OLAP Functions
The OLAP functions extend the syntax of the SQL analytic functions. 
Using these OLAP functions, you can create all standard calculated measures, including rank, share, prior and future periods, period-to-date, parallel period, moving aggregates, and cumulative aggregates.
Commands are grouped into these categories:

- Aggregate Functions
- Analytic Functions
- Hierarchical Functions
- Lag Functions
- OLAP DML Functions
- Rank Functions
- Share Functions
- Window Functions


## Aggregate Functions

### AVG
`AVG` returns the average of a selection of values calculated over a dimension.

In [26]:
duckdb.sql("""
           
SELECT
       Type,
       AVG(Expenses) AS AVG_expenses
FROM df
GROUP BY 1
ORDER BY 2 DESC
           
    """)

┌─────────┬──────────────┐
│  Type   │ AVG_expenses │
│ varchar │    double    │
├─────────┼──────────────┤
│ Gaming  │        100.0 │
│ Book    │        100.0 │
│ Gift    │        81.75 │
│ Clothes │       19.625 │
│ Coffee  │        2.875 │
└─────────┴──────────────┘

### COUNT
`COUNT` tallies the number of data values identified by a selection of members in any dimension.

In [27]:
duckdb.sql("""
           
SELECT
       Type,
       COUNT(*) AS num_records
FROM df
GROUP BY 1
ORDER BY 2 DESC
           
    """)

┌─────────┬─────────────┐
│  Type   │ num_records │
│ varchar │    int64    │
├─────────┼─────────────┤
│ Clothes │           4 │
│ Gift    │           2 │
│ Gaming  │           2 │
│ Coffee  │           2 │
│ Book    │           1 │
└─────────┴─────────────┘

### MAX & MIN
- `MAX`returns the largest value of a selection of data values calculated over a dimension.
- `MIN`returns the smallest value of a selection of data values calculated over a dimension.

In [30]:
duckdb.sql("""
           
SELECT
       Type,
       MAX(Expenses) AS max_expense,
       MIN(Expenses) AS min_expense,
FROM df
GROUP BY 1
ORDER BY 1
           
    """)

┌─────────┬─────────────┬─────────────┐
│  Type   │ max_expense │ min_expense │
│ varchar │   double    │   double    │
├─────────┼─────────────┼─────────────┤
│ Book    │       100.0 │       100.0 │
│ Clothes │        25.0 │         5.0 │
│ Coffee  │        3.75 │         2.0 │
│ Gaming  │       100.0 │       100.0 │
│ Gift    │       143.5 │        20.0 │
└─────────┴─────────────┴─────────────┘

### SUM
`SUM` returns the total of a selection of values calculated over a Time dimension. 

In [31]:
duckdb.sql("""
           
SELECT
       Type,
       SUM(Expenses) AS total_expenses,
FROM df
GROUP BY 1
ORDER BY 1
           
    """)

┌─────────┬────────────────┐
│  Type   │ total_expenses │
│ varchar │     double     │
├─────────┼────────────────┤
│ Book    │          100.0 │
│ Clothes │           78.5 │
│ Coffee  │           5.75 │
│ Gaming  │          100.0 │
│ Gift    │          163.5 │
└─────────┴────────────────┘

### RANK
`RANK` orders the members of a dimension based on the values of an expression. The function returns the sequence numbers of the dimension members.

In [32]:
# Get the top-3 busiest train stations
duckdb.sql("""
           
SELECT *, 
       RANK() OVER (ORDER BY Expenses DESC) AS Rank,
FROM df
           
    """)

┌─────────────────────┬──────────┬─────────┬───────┬───────┬───────┬───────┐
│        Date         │ Expenses │  Type   │  Day  │ Month │ Year  │ Rank  │
│    timestamp_ns     │  double  │ varchar │ int32 │ int32 │ int32 │ int64 │
├─────────────────────┼──────────┼─────────┼───────┼───────┼───────┼───────┤
│ 2022-01-01 00:00:00 │    143.5 │ Gift    │     1 │     1 │  2022 │     1 │
│ 2022-01-07 00:00:00 │    100.0 │ Gaming  │     7 │     1 │  2022 │     2 │
│ 2022-01-10 00:00:00 │    100.0 │ Book    │    10 │     1 │  2022 │     2 │
│ 2022-01-01 00:00:00 │     25.0 │ Clothes │     1 │     1 │  2022 │     4 │
│ 2022-01-08 00:00:00 │     24.5 │ Clothes │     8 │     1 │  2022 │     5 │
│ 2022-01-01 00:00:00 │     24.0 │ Clothes │     1 │     1 │  2022 │     6 │
│ 2022-01-05 00:00:00 │     20.0 │ Gift    │     5 │     1 │  2022 │     7 │
│ 2022-01-06 00:00:00 │      5.0 │ Clothes │     6 │     1 │  2022 │     8 │
│ 2022-01-02 00:00:00 │     3.75 │ Coffee  │     2 │     1 │  2022 │     9 │

## Lag Functions
`LAG` retrieves data from previous rows over the columns we indicate. 


In [15]:
# Get the top-3 busiest train stations
duckdb.sql("""
           
SELECT *,
        LAG(Expenses) OVER(PARTITION BY Type ORDER BY Date)
FROM df
           
    """)

┌─────────────────────┬──────────┬─────────┬───────┬───────┬───────┬───────────────────────────────────────────────────┐
│        Date         │ Expenses │  Type   │  Day  │ Month │ Year  │ lag(Expenses) OVER (PARTITION BY "Type" ORDER B…  │
│    timestamp_ns     │  double  │ varchar │ int32 │ int32 │ int32 │                      double                       │
├─────────────────────┼──────────┼─────────┼───────┼───────┼───────┼───────────────────────────────────────────────────┤
│ 2022-01-01 00:00:00 │     NULL │ Gaming  │     1 │     1 │  2022 │                                              NULL │
│ 2022-01-07 00:00:00 │    100.0 │ Gaming  │     7 │     1 │  2022 │                                              NULL │
│ 2022-01-01 00:00:00 │    143.5 │ Gift    │     1 │     1 │  2022 │                                              NULL │
│ 2022-01-05 00:00:00 │     20.0 │ Gift    │     5 │     1 │  2022 │                                             143.5 │
│ 2022-01-10 00:00:00 │    100.0

## Rank Functions
The rank functions numerate the records of a table with a specific order. The way they deal with ties makes us have three different commands to do so:
- `RANK`Deals with ties, giving the same number to records with the same value, leaving gaps after them. 
- `ROW_NUMBER` Give unique numbers to all records. If the value is the same, it just gives a random order. 
- `DENSE_RANK` Deals with ties, giving the same number to records with the same value, without leaving gaps after them. 

In [14]:
# Get the top-3 busiest train stations
duckdb.sql("""
           
SELECT *, 
       RANK() OVER (ORDER BY Expenses DESC) AS Rank,
       ROW_NUMBER() OVER (ORDER BY Expenses DESC) AS Row_Number,
       DENSE_RANK() OVER (ORDER BY Expenses DESC) AS Dense_Rank
FROM df
           
    """)

┌─────────────────────┬──────────┬─────────┬───────┬───────┬───────┬───────┬────────────┬────────────┐
│        Date         │ Expenses │  Type   │  Day  │ Month │ Year  │ Rank  │ Row_Number │ Dense_Rank │
│    timestamp_ns     │  double  │ varchar │ int32 │ int32 │ int32 │ int64 │   int64    │   int64    │
├─────────────────────┼──────────┼─────────┼───────┼───────┼───────┼───────┼────────────┼────────────┤
│ 2022-01-01 00:00:00 │    143.5 │ Gift    │     1 │     1 │  2022 │     1 │          1 │          1 │
│ 2022-01-07 00:00:00 │    100.0 │ Gaming  │     7 │     1 │  2022 │     2 │          2 │          2 │
│ 2022-01-10 00:00:00 │    100.0 │ Book    │    10 │     1 │  2022 │     2 │          3 │          2 │
│ 2022-01-01 00:00:00 │     25.0 │ Clothes │     1 │     1 │  2022 │     4 │          4 │          3 │
│ 2022-01-08 00:00:00 │     24.5 │ Clothes │     8 │     1 │  2022 │     5 │          5 │          4 │
│ 2022-01-01 00:00:00 │     24.0 │ Clothes │     1 │     1 │  2022 │     

# 2. SQL ROW FUNCTIONS
The OLAP row functions extend the syntax of the SQL row functions for use with dimensional objects. 
If you use the SQL row functions, then this syntax is familiar. 
You can use these functions on relational data when loading it into cubes and cube dimensions, and with the OLAP functions when creating calculated measures.

This chapter describes the row functions of the OLAP expression syntax.

The row functions are grouped into the following categories:

- Numeric Functions
- Character Functions That Return Characters
- NLS Character Functions
- Character Functions That Return Numbers
- Datetime Functions
- General Comparison Functions
- Conversion Functions
- Encoding and Decoding Function
- Null-Related Functions
- Environment and Identifier Functions

In process of being constructed! 🏗️

## Null-Related Functions
These functions facilitate null handling:

### COALESCE
`COALESCE` returns the first non-null expression in a list.

In [6]:
# Get the top-3 busiest train stations
import duckdb
duckdb.sql("""
           
    SELECT 
           *,
           COALESCE(Expenses, 0) AS Expenses_mod
    FROM df
           
    """)

┌─────────────────────┬──────────┬─────────┬───────┬───────┬───────┬──────────────┐
│        Date         │ Expenses │  Type   │  Day  │ Month │ Year  │ Expenses_mod │
│    timestamp_ns     │  double  │ varchar │ int32 │ int32 │ int32 │    double    │
├─────────────────────┼──────────┼─────────┼───────┼───────┼───────┼──────────────┤
│ 2022-01-01 00:00:00 │    143.5 │ Gift    │     1 │     1 │  2022 │        143.5 │
│ 2022-01-01 00:00:00 │     25.0 │ Clothes │     1 │     1 │  2022 │         25.0 │
│ 2022-01-01 00:00:00 │     NULL │ Gaming  │     1 │     1 │  2022 │          0.0 │
│ 2022-01-01 00:00:00 │     24.0 │ Clothes │     1 │     1 │  2022 │         24.0 │
│ 2022-01-02 00:00:00 │     3.75 │ Coffee  │     2 │     1 │  2022 │         3.75 │
│ 2022-01-05 00:00:00 │     20.0 │ Gift    │     5 │     1 │  2022 │         20.0 │
│ 2022-01-06 00:00:00 │      5.0 │ Clothes │     6 │     1 │  2022 │          5.0 │
│ 2022-01-07 00:00:00 │    100.0 │ Gaming  │     7 │     1 │  2022 │        